In [ ]:
import numpy as np
import pandas as pd
import datatable as dt

In [ ]:
# load train dataset
X = pd.read_csv('../input/tps-apr-22-fe-v2/train_tps_apr_22_fev2.csv')

y = X[['state']]
groups = X['sequence']
X = X.drop(['sequence', 'subject'], inplace=False, axis=1)

# LightGBMError: Do not support special JSON characters in feature name.
import re
X = X.rename(columns = lambda x:re.sub('[^A-Za-z0-9_]+', '', x))

In [ ]:
from sklearn.model_selection import StratifiedKFold, GroupKFold
from sklearn.metrics import roc_curve, auc, roc_auc_score
import optuna

def objective(trial, data=X, target=y):
    # X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    # GroupKFold split once
    # X_train, X_test, y_train, y_test = kf.split(X, y, groups.unique())
    
    params = {
        'metric': 'auc', 
        'random_state': 42,
        'device': 'gpu',
        'gpu_platform_id': 0,
        'gpu_device_id': 0,
        'n_estimators': trial.suggest_int('n_estimators', 500, 20000, 500),
        'learning_rate': trial.suggest_float('learning_rate', 0.005, 0.1),
        'reg_alpha': trial.suggest_loguniform('reg_alpha', 1e-3, 10.0),
        'reg_lambda': trial.suggest_loguniform('reg_lambda', 1e-3, 10.0),
        'colsample_bytree': trial.suggest_categorical('colsample_bytree', [0.3,0.4,0.5,0.6,0.7,0.8,0.9,1.0]),
        'subsample': trial.suggest_categorical('subsample', [0.4,0.5,0.6,0.7,0.8,1.0]),
        'max_depth': trial.suggest_categorical('max_depth', [10,20,100]),
        'num_leaves' : trial.suggest_int('num_leaves', 1, 1000),
        'min_child_samples': trial.suggest_int('min_child_samples', 1, 300),
        'cat_smooth' : trial.suggest_int('min_data_per_groups', 1, 100)
    }
    
    # kf = GroupKFold(n_splits=3)
    oof = np.zeros(len(X))
    idx1 = X.index
    
    gkf = GroupKFold(n_splits=3)
    
    for train_idx, valid_idx in gkf.split(X, y, groups.unique()):
        # split train and label into X, y train and valid set.
        X_train, y_train = X.loc[train_idx,:], y.loc[train_idx,'state']
        X_valid, y_valid = X.loc[valid_idx,:], y.loc[valid_idx,'state']
        
        model = LGBMClassifier(**params)  
        model.fit(X_train, y_train, eval_set = [(X_valid, y_valid)], 
                  early_stopping_rounds=300, verbose=False)
        oof[idx1[valid_idx]] = model.predict_proba(X.loc[valid_idx,:])[:,1]
    
    auc = roc_auc_score(y, oof)
    
    return auc

In [ ]:
from lightgbm import LGBMClassifier

study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=100, gc_after_trial=True)
print('Number of finished trials:', len(study.trials))
print('Best trial:', study.best_trial.params)

In [ ]:
# optuna auc == 1.0 ???
# {'n_estimators': 16500, 'learning_rate': 0.06946446628215118, 'reg_alpha': 0.010253222571604808, 'reg_lambda': 0.031961179402523146, 'colsample_bytree': 1.0, 'subsample': 0.6, 'max_depth': 100, 'num_leaves': 867, 'min_child_samples': 291, 'min_data_per_groups': 26}